### 문제 1-1 : 기본 체인 만들기 - AI 요리사

In [1]:
import os
from dotenv import load_dotenv 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [17]:
prompt_template = PromptTemplate(
    input_variables=["ingredients"],
    template="""
    다음 재료들로 만들 수 있는 요리를 하나만 추천해주세요: {ingredients}.
    각 요리에 대해 아래 내용을 포함해주세요:
    추천 요리 이름
    입력한 재료 목록
    간단한 조리법(조리법은 순서를 매겨주세요.)
    """,
)

chain = prompt_template | llm | StrOutputParser()

In [18]:
try:
    result = chain.invoke({"ingredients":"계란, 밥, 김치"})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

### **김치 계란밥**

*   **재료:** 계란, 밥, 김치
*   **조리법:**

    1.  밥을 준비합니다. 
    2.  김치를 잘게 썰어줍니다. 
    3.  계란을 풀어줍니다. 
    4.  프라이팬에 기름을 두르고 썰어놓은 김치를 볶아줍니다. 
    5.  김치가 볶아지면 풀어놓은 계란을 부어줍니다. 
    6.  계란이 반숙 정도 익으면 다 익지 않은 밥을 넣고 볶아줍니다. 
    7.  모든 재료가 섞이고 밥이 잘 볶아지면 맛있는 김치 계란밥이 됩니다.


### 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추천한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요. 각 문장은 번호를 매기어 줄 바꿈을 해주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [22]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

추천한 영화의 제목은 '매트릭스'입니다.

1. 영화 '매트릭스'는 가상 현실 세계인 매트릭스에서 살아가는 인류의 이야기입니다. 
2. 주인공 '네오'는 컴퓨터 해커로, '모피어스'라는 인물에게 매트릭스의 진실을 알게 되고, 가상 현실에서 벗어나기 위한 선택을 하게 됩니다.
3. 네오는 매트릭스로부터 인류를 구하기 위해 '머피어스'와 함께 싸우게 되며, 이 과정에서 혁신적인 시각 효과와 철학적 주제로 유명해졌습니다.


### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [23]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "애플이 아이폰을 비롯한 자사 기기 운영체제(OS)에 입체감을 살린 ‘반투명 디자인’을 도입하며 대대적인 개편을 예고했다. 아이폰 운영체제 iOS7을 출시한 2013년 이후 가장 큰 규모의 디자인 개편이다.",
        "keywords": "애플, 디자인, 개편"
    },
]


# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

examples = [
    {
        "news": "삼성전자가 AI 가속기를 출시할 예정이다.",
        "keywords": "삼성전자, AI가속기, 출시"
    },
    {
        "news": "WHO가 국제 협력의 중요성을 강조했다.",
        "keywords": "WHO, 국제협력, 보건"
    },
    {
        "news": "애플이 디자인 개편을 예고했다.",
        "keywords": "애플, 디자인, 개편"
    },
]


# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])


# 모델 생성 및 체인 구성
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

키워드: 제미나이, 구글, AI
